# Startup

To begin a django project, run `django-admin startproject <name of project>`. To run the project, run `python manage.py runserver` and visit `localhost:8000` or `127.0.0.1:8000`. Note that the default port is `8000`. 

Django projects can have multiple applications to handle various aspects (Blog, Wiki, Forums). To create an application, run `python manage.py startapp <name of application>`. The application will need to be added to `INSTALLED_APPS` in `settings.py`.

# Architecture

This is the overall architecture of a Django applicaiton.

- Project
    - settings.py (configures the project)
    - urls.py (provides URL routing)
    - wsgi.py (provides web-hosting hooks for Apache, nginx, etc.)
- Application
    - admin.py (handles the logic for application oversight)
    - apps.py (configures the application)
    - models.py (creates database tables, manages queries)
    - views.py (provides the response for the HTTP/S requests; serves the HTML/JS payloads)

Django follows the *model-view-controller* logic. An HTTP request flows directly to `urls.py`, which maps the endpoint to a function (*a view*). Generally, custom function is imported from `views.py`. The function takes in the `request` and provides an HTML `response`, possible by interfacing with data created/updated/deleted in using `models.py` and/or picking up HTML/JS payloads from `templates` or `static` directories.  

# Model

Some models.py field types include `CharField`, `TextField`, `EmailField`, `URLField`, `IntegerField`, `DecimalField`, `BooleanField`, and `DateTimeField`. One can use `ForeignKey` or `ManyToMany` attributes to link entries.

An example of `models.py` is the following.
```python

from django.db import models

class Pet(models.Model):
    SEX_CHOICES = [('M', 'Male'), ('F', 'Female')]
    name = models.CharField(max_length=100)
    submitter = models.CharField(max_length=100)
    species = models.CharField(max_length=30)
    breed = models.CharField(max_length=30, blank=True)
    description = models.TextField()
    sex = models.CharField(max_length=1, choices=SEX_CHOICES, blank=True)
    submission_date = models.DateTimeField()
    age = models.IntegerField(null=True)
    vaccinations = models.ManyToManyField('Vaccine', blank=True)

class Vaccine(models.Model):
    name = models.CharField(max_length=50)
```

For making changes to the database, run `python manage.py makemigrations`, `python manage.py showmigrations`, and `python manage.py migrate <app name> <number>`.

One can create their own `manage.py` custom commands by creating a file under `/project/app/management/command/<file.py>`. An example code block (it's ran by typing `python manage.py <file.py>`):
```python
from csv import DictReader
from datetime import datetime

from django.core.management import BaseCommand

from adoptions.models import Pet, Vaccine
from pytz import UTC


DATETIME_FORMAT = '%m/%d/%Y %H:%M'

VACCINES_NAMES = [
    'Canine Parvo',
    'Canine Distemper',
    'Canine Rabies',
    'Canine Leptospira',
    'Feline Herpes Virus 1',
    'Feline Rabies',
    'Feline Leukemia'
]

ALREDY_LOADED_ERROR_MESSAGE = """
If you need to reload the pet data from the CSV file,
first delete the db.sqlite3 file to destroy the database.
Then, run `python manage.py migrate` for a new empty
database with tables"""


class Command(BaseCommand):
    # Show this when the user types help
    help = "Loads data from pet_data.csv into our Pet mode"

    def handle(self, *args, **options):
        if Vaccine.objects.exists() or Pet.objects.exists():
            print('Pet data already loaded...exiting.')
            print(ALREDY_LOADED_ERROR_MESSAGE)
            return
        print("Creating vaccine data")
        for vaccine_name in VACCINES_NAMES:
            vac = Vaccine(name=vaccine_name)
            vac.save()
        print("Loading pet data for pets available for adoption")
        for row in DictReader(open('./pet_data.csv')):
            pet = Pet()
            pet.name = row['Pet']
            pet.submitter = row['Submitter']
            pet.species = row['Species']
            pet.breed = row['Breed']
            pet.description = row['Pet Description']
            pet.sex = row['Sex']
            pet.age = row['Age']
            raw_submission_date = row['submission date']
            submission_date = UTC.localize(
                datetime.strptime(raw_submission_date, DATETIME_FORMAT))
            pet.submission_date = submission_date
            pet.save()
            raw_vaccination_names = row['vaccinations']
            vaccination_names = [name for name in raw_vaccination_names.split('| ') if name]
            for vac_name in vaccination_names:
                vac = Vaccine.objects.get(name=vac_name)
                pet.vaccinations.add(vac)
            pet.save()
```

# Admin

To create a new superuser for to access the admin page, run `python manage.py createsuperuser` and interact with the prompt. The `/admin/` endpoint provides an out-of-the-box page. 

Run `python manage.py shell` to get a Python session with Django (it's helpful to interface with the ORM). For the example above, a common query is `Pet.objects.all()` or `Pet.objects.get(id=<number>)`.

# URL Routing

Examples for routing performed in `urls.py` is shown here:

Example 1:
```python
urlpatterns = [
    path(r'admin/', admin.site.urls),
    url(r'^$', Views.home, name='home')
]
```

Example 2:
```python
urlpatterns = [
    path('admin/', admin.site.urls),
    path('', views.home, name='home'),
    path('adoptions/<int:pet_id>/', views.pet_detail, name='pet_detail')
]
```

Note that `Example 2` uses the path converted `<int:pet_id>` to dynamically take in arguments from the URL. These in turn can be used to query the database for specific items.

# Views

Most of the custom-developed software will live in `views.py`, which serves to handle the requests and produce an app-level response. Using the sample Python code above, a sample code block would be:
```python
from django.shortcuts import render
from django.http import Http404

from .models import Pet

def home(request):
    pets = Pet.objects.all()
    return render(request, 'home.html', {
        'pets': pets,
    })

def pet_detail(request, pet_id):
    try:
        pet = Pet.objects.get(id=pet_id)
    except Pet.DoesNotExist:
        raise Http404('pet not found')
    return render(request, 'pet_detail.html', {
        'pet': pet,
    })
```

# Templates

## Jinja Templating

When rendering HTML, one can use templating to passing variables from `views.py` into their `templates/` files. Using `{{ }}` for variables and `{% %}` for looping/conditionals. The `|` is used to apply either built-in filters or custom-built filters.

A `home.html` example is:
```html
{% extends "base.html" %}
{% block content %}
<div>
    {% for pet in pets %}
        <div class="petname">
            <a href="{% url 'pet_detail' pet.id %}">
            <h3>{{ pet.name|capfirst }}</h3>
            </a>
            <p>{{ pet.species }}</p>
            {% if pet.breed %}
                <p>Breed: {{ pet.breed }}</p>
            {% endif %}
            <p class="hidden">{{ pet.description }}</p>
        </div>
    {% endfor %}
</div>
{% endblock %}
```
while a `pet_detail` example is:
```html
{% extends "base.html" %}
{% block content %}
<div>
    <h3>{{ pet.name|capfirst }}</h3>
    <p>{{ pet.species }}</p>
    {% if pet.breed %}
        <p>Breed: {{ pet.breed }}</p>
    {% endif %}
    {% if pet.age %}
        <p>Age: {{ pet.age }}</p>
    {% endif %}
    {% if pet.sex %}
        <p>Sex: {{ pet.sex }}</p>
    {% endif %}
    {% if pet.vaccinations.all %}
        <p>Vaccinations for:</p>
        <ul>
            {% for vaccination in pet.vaccinations.all %}
                <li>{{ vaccination.name }}</li>
            {% endfor %}
        </ul>
    {% endif %}
    <p>Submitted by: {{ pet.submitter }}</p>
    <p>Submitted on: {{ pet.submission_date|date:"M d Y" }}</p>
    <p>{{ pet.description }}</p>
</div>
{% endblock %}
```